In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from numpy import mean
from numpy import std
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV

import keras.backend as K
from keras import regularizers
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import RMSprop
from keras.optimizers import Adam

In [ ]:
# link to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def initialise(data):
  data.index = data["date"]
  data.index = pd.to_datetime(data.index)
  data.drop(columns=["date"], inplace=True)
  return data

data_level = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_level_mapped_selection10.csv'))
data_slope = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_slope_mapped_selection10.csv'))
data_curve = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_curve_mapped_selection10.csv'))

data_level.head()

,secid,-80,-50,-20,level,slope,curve,permno,beta,betasq,...,cash,cinvest,roaq,roavol,ms,baspread,maxret,retvol,std_dolvol,std_turn
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,100862,0.307688,0.315627,0.358095,0.315627,0.042468,0.017264,54594,-0.502126,-0.886445,...,-0.868511,0.071798,0.396279,-0.953113,0.00,-0.689049,-0.662673,-0.619551,0.327234,-0.936622
1996-01-31,100871,0.705411,0.615432,0.651160,0.615432,0.035728,0.062854,50906,0.551365,0.193703,...,-0.738289,0.080067,0.538981,-0.900059,0.75,-0.112575,-0.390130,-0.223204,-0.342296,-0.748948
1996-01-31,100892,0.270090,0.216768,0.216508,0.216768,-0.000260,0.026531,57904,-0.536730,-0.902575,...,-0.958006,0.066337,0.367446,-0.998951,0.25,-0.786859,-0.947305,-0.877289,-0.536138,-0.955821
1996-01-31,100896,0.430796,0.436291,0.429517,0.436291,-0.006774,-0.006135,77520,0.349363,-0.101800,...,-0.968644,0.060739,0.459243,-0.859083,-0.25,-0.747736,-0.649524,-0.703492,-0.331142,-0.818216
1996-01-31,100903,0.262405,0.252683,0.373839,0.252683,0.121156,0.065439,80303,-0.186662,-0.682642,...,-0.584473,0.077826,0.605842,-0.875346,-0.50,-0.807682,-0.825541,-0.737079,-0.400503,-0.866238


In [ ]:
def samplesplitting(data,train,validate,oos,i):
    train_data    = data[(data.index.year >= 1996 )         & (data.index.year < i - validate)]
    validate_data = data[(data.index.year >= i - validate)  & (data.index.year < i)]
    total_train   = data[(data.index.year >= 1996 )         & (data.index.year < i)]
    oos_data      = data[(data.index.year == i)]
    pred_data     = data[(data.index.year >= 1996 + train + validate) & (data.index.year <= i)]

    print('train=%d, validate=%d, oos=%d' % (len(train_data), len(validate_data), len(oos_data)))
    return train_data, validate_data, oos_data, total_train, pred_data

def split_xy(data, kind):
  data_x = data.iloc[:,8:]
  data_y = data[kind]
  return data_x, data_y

# Level

In [ ]:
# Complete number of observations is 982285
# We take the initial train, validate and oos sample to be 8, 5 and 13 years respectively (Ratios similar to Gu Kelly Xiu)
# On average one month contains 3000 options, therefore we loop over 12 * 3000 options, which is similar to looping over a year
trainsize = 11 * 2000 * 12
validatesize = 5 * 2000 * 12
oossize = 2000 * 12

# Initialise predictionstring
IV = 'level'
oos_pred_level = pd.DataFrame()

# define the model
def create_model(bias, activity):
  model = Sequential()
  model.add(Dense(units=32,
                  activation='relu',
                  input_dim= train_x.shape[1],
                  # # kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-5),
                  bias_regularizer= regularizers.L2(bias),
                  activity_regularizer= regularizers.L2(activity)
                  ))
  model.add(Dense(units=32, activation= 'relu',
                  bias_regularizer=regularizers.L2(bias),
                  activity_regularizer=regularizers.L2(activity) #activation
                  ))
  model.add(Dense(units=1, activation= 'linear')) #miss linear

  # compile the model
  model.compile(optimizer= RMSprop(learning_rate=0.1), #'rmsprop',#Optimizer_trial,
                loss= 'mean_squared_error',
                metrics=['mse'])
  return model

window = 9
train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
train_x, train_y              = split_xy(train,IV)
validate_x, validate_y        = split_xy(validate,IV)
oos_x, oos_y                  = split_xy(oos,IV)
total_train_x, total_train_y  = split_xy(total_train,IV)

Parameter_Trials={'batch_size':[100],
                      'epochs':[50],
                      'bias':  [1e-4] ,
                      'activity':  [1e-6]
                    #'Optimizer_trial':['rmsprop'], #'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'
                  #'activation': ['linear']#['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
                  # 'units1': [64, 32],
                  # 'units2': [32, 16]
                 } #learn rate, momentum

RegModel=KerasRegressor(create_model, verbose=0)
grid_search=GridSearchCV(estimator=RegModel, param_grid=Parameter_Trials, cv=2, verbose = 10)
model_final = grid_search.fit(total_train_x, total_train_y)
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

for window in range (9,10):
  print('Iteration: ', window)

  # Sample splitting
  train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  # predictions = pd.DataFrame(index=range(10), columns=['score', 'oos_prediction'])

  # for network in range(0,10):
  #   model=KerasRegressor(create_model, verbose=0)
  #   model.fit(train_x, train_y, batch_size=5, epochs=30)

  #   pred_val = model.predict(validate_x)
  #   y_hat_oos = model.predict(oos_x)
  #   print(network)
  #   print('R^2: %.8f' % (r2_score(validate_y, pred_val)))
  #   print('MSE: %.8f' % (mean_squared_error(validate_y, pred_val)))
  #   predictions.iloc[network,:] = [mean_squared_error(y_validate, pred_val), y_hat_oos]

  # sorted_predictions = predictions.sort_values(by='score', ascending=True)
  # y_hat = pd.DataFrame(mean(sorted_predictions.iloc[0:3,1]))
  # y_hat.index = oos_y.index

  # Fit model
  # print('Nr of trees: ', n_estimators_final)
  # print('Max Depth: ', max_depth_final)
  # print('Max Features: ', max_features_final)
  # model_final = RandomForestRegressor(n_estimators=n_estimators_final,
  #                                    max_depth=max_depth_final,
  #                                    max_features=max_features_final)
  # model_final.fit(total_train_x, total_train_y)
  y_hat = pd.DataFrame(model_final.predict(oos_x))
  print('Yearly:',r2_score(oos_y, y_hat))
  oos_pred_level = pd.concat([oos_pred_level, y_hat], ignore_index=True) #oos_pred_level = oos_pred_level.append(y_hat)
  # Optimisation method 2 GridSearch

oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/level_predictions_NN_tuning9.csv', index=True)


train=408000, validate=120000, oos=24000
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2; 1/1] START activity=1e-06, batch_size=100, bias=0.0001, epochs=50......


<ipython-input-7-f6b829329de5>:51: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  RegModel=KerasRegressor(create_model, verbose=0)


[CV 1/2; 1/1] END activity=1e-06, batch_size=100, bias=0.0001, epochs=50;, score=-0.042 total time= 2.7min
[CV 2/2; 1/1] START activity=1e-06, batch_size=100, bias=0.0001, epochs=50......
[CV 2/2; 1/1] END activity=1e-06, batch_size=100, bias=0.0001, epochs=50;, score=-0.066 total time= 2.7min
Best Hyperparameters: {'activity': 1e-06, 'batch_size': 100, 'bias': 0.0001, 'epochs': 50}
Iteration:  9
train=408000, validate=120000, oos=24000
Yearly: 0.3068934350927355


In [ ]:
# Complete number of observations is 982285
# We take the initial train, validate and oos sample to be 8, 5 and 13 years respectively (Ratios similar to Gu Kelly Xiu)
# On average one month contains 3000 options, therefore we loop over 12 * 3000 options, which is similar to looping over a year
trainsize = 8 * 2000 * 12
validatesize = 5 * 2000 * 12
oossize = 2000 * 12

# Initialise predictionstring
IV = 'level'
oos_pred_level = pd.DataFrame()

# #start of last prediction
# def isolate(data):
#   data = data.iloc[:,1:]
#   return data
# oos_pred_level   = isolate(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/level_predictions_NN_1_progress.csv'))
# oos_pred_level.columns = ['Predictions']

# define the model
def create_model():
  model = Sequential()
  model.add(Dense(units=32,
                  activation='relu',
                  input_dim= train_x.shape[1],
                  bias_regularizer= regularizers.L2(1e-4),
                  activity_regularizer= regularizers.L2(1e-6)
                  ))
  model.add(Dense(units=32, activation= 'relu',
                  bias_regularizer=regularizers.L2(1e-4),
                  activity_regularizer=regularizers.L2(1e-6) #activation
                  ))
  model.add(Dense(units=1, activation= 'linear')) #miss linear

  # compile the model
  model.compile(optimizer= Adam(), # was 0.01
                loss= 'mean_squared_error',
                metrics=['mse'])
  return model

# window = 9
# train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
# train_x, train_y              = split_xy(train,IV)
# validate_x, validate_y        = split_xy(validate,IV)
# oos_x, oos_y                  = split_xy(oos,IV)
# total_train_x, total_train_y  = split_xy(total_train,IV)

# Parameter_Trials={'batch_size':[100],
#                       'epochs':[50],
#                       'bias':  [1e-4] ,
#                       'activity':  [1e-6]
                    #'Optimizer_trial':['rmsprop'], #'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'
                  #'activation': ['linear']#['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
                  # 'units1': [64, 32],
                  # 'units2': [32, 16]
                #  } #learn rate, momentum

# RegModel=KerasRegressor(create_model, verbose=0)
# grid_search=GridSearchCV(estimator=RegModel, param_grid=Parameter_Trials, cv=2, verbose = 10)
# model_final = grid_search.fit(total_train_x, total_train_y)
# best_params = grid_search.best_params_
# print("Best Hyperparameters:", best_params)

model_final = KerasRegressor(create_model, epochs=50, batch_size=100, verbose=0)

for window in range (0,10):
  print('Iteration: ', window)

  # Sample splitting
  train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  predictions = pd.DataFrame(index=range(10), columns=['score', 'oos_prediction'])

  for network in range(0,10):
    # model=KerasRegressor(create_model, verbose=0)
    model_final.fit(total_train_x, total_train_y)

    pred_val = model_final.predict(validate_x)
    y_hat_oos = model_final.predict(oos_x)
    print(network)
    print('R^2: %.8f' % (r2_score(validate_y, pred_val)))
    print('MSE: %.8f' % (mean_squared_error(validate_y, pred_val)))
    predictions.iloc[network,:] = [mean_squared_error(validate_y, pred_val), y_hat_oos]

  sorted_predictions = predictions.sort_values(by='score', ascending=True)
  y_hat = pd.DataFrame(mean(sorted_predictions.iloc[0:3,1]))
  y_hat.index = oos_y.index
  y_hat.columns = ['Predictions']

  # Fit model
  # print('Nr of trees: ', n_estimators_final)
  # print('Max Depth: ', max_depth_final)
  # print('Max Features: ', max_features_final)
  # model_final = RandomForestRegressor(n_estimators=n_estimators_final,
  #                                    max_depth=max_depth_final,
  #                                    max_features=max_features_final)
  # model_final.fit(total_train_x, total_train_y)
  # y_hat = pd.DataFrame(model_final.predict(oos_x))

  print('Yearly:',r2_score(oos_y, y_hat))
  oos_pred_level = pd.concat([oos_pred_level, y_hat], ignore_index=True) #oos_pred_level = oos_pred_level.append(y_hat)
  print('Total:',r2_score(total_oos[IV], oos_pred_level))
  oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/level_predictions_NN_1_progress.csv', index=True)

oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/level_predictions_NN_1.csv', index=True)


<ipython-input-5-9faa668deb4e>:63: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_final = KerasRegressor(create_model, epochs=50, batch_size=100, verbose=0)


Iteration:  0
train=192000, validate=120000, oos=24000
0
R^2: 0.62654228
MSE: 0.02501595
1
R^2: 0.59990210
MSE: 0.02680043
2
R^2: 0.60821373
MSE: 0.02624368
3
R^2: 0.59730815
MSE: 0.02697419
4
R^2: 0.61775214
MSE: 0.02560475
5
R^2: 0.60763638
MSE: 0.02628235
6
R^2: 0.61400094
MSE: 0.02585603
7
R^2: 0.61214205
MSE: 0.02598054
8
R^2: 0.60199290
MSE: 0.02666038
9
R^2: 0.60493025
MSE: 0.02646362
Yearly: 0.3147790756908757
Total: 0.3147790756908757
Iteration:  1
train=216000, validate=120000, oos=24000
0
R^2: 0.58209944
MSE: 0.02855618
1
R^2: 0.57367586
MSE: 0.02913178


In [ ]:
oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/RF/level_predictions_RF.csv', index=True)

# Slope

In [ ]:
# Complete number of observations is 982285
# We take the initial train, validate and oos sample to be 8, 5 and 13 years respectively (Ratios similar to Gu Kelly Xiu)
# On average one month contains 3000 options, therefore we loop over 12 * 3000 options, which is similar to looping over a year
trainsize = 11 * 2000 * 12
validatesize = 5 * 2000 * 12
oossize = 2000 * 12

# Initialise predictionstring
IV = 'slope'
oos_pred_slope = pd.DataFrame()

# #start of last prediction
# def isolate(data):
#   data = data.iloc[:,1:]
#   return data
# oos_pred_slope   = isolate(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/slope_predictions_NN_1_progress.csv'))
# oos_pred_slope.columns = ['Predictions']

# define the model
def create_model():
  model = Sequential()
  model.add(Dense(units=32,
                  activation='relu',
                  input_dim= train_x.shape[1],
                  bias_regularizer= regularizers.L2(1e-4),
                  activity_regularizer= regularizers.L2(1e-6)
                  ))
  model.add(Dense(units=32, activation= 'relu',
                  bias_regularizer=regularizers.L2(1e-4),
                  activity_regularizer=regularizers.L2(1e-6) #activation
                  ))
  model.add(Dense(units=1, activation= 'linear')) #miss linear

  # compile the model
  model.compile(optimizer= Adam(), # was 0.01
                loss= 'mean_squared_error',
                metrics=['mse'])
  return model

# window = 9
# train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
# train_x, train_y              = split_xy(train,IV)
# validate_x, validate_y        = split_xy(validate,IV)
# oos_x, oos_y                  = split_xy(oos,IV)
# total_train_x, total_train_y  = split_xy(total_train,IV)

# Parameter_Trials={'batch_size':[100],
#                       'epochs':[50],
#                       'bias':  [1e-4] ,
#                       'activity':  [1e-6]
                    #'Optimizer_trial':['rmsprop'], #'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'
                  #'activation': ['linear']#['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
                  # 'units1': [64, 32],
                  # 'units2': [32, 16]
                #  } #learn rate, momentum

# RegModel=KerasRegressor(create_model, verbose=0)
# grid_search=GridSearchCV(estimator=RegModel, param_grid=Parameter_Trials, cv=2, verbose = 10)
# model_final = grid_search.fit(total_train_x, total_train_y)
# best_params = grid_search.best_params_
# print("Best Hyperparameters:", best_params)

model_final = KerasRegressor(create_model, epochs=50, batch_size=100, verbose=0)

for window in range (0,10):
  print('Iteration: ', window)

  # Sample splitting
  train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  predictions = pd.DataFrame(index=range(10), columns=['score', 'oos_prediction'])

  for network in range(0,10):
    # model=KerasRegressor(create_model, verbose=0)
    model_final.fit(total_train_x, total_train_y)

    pred_val = model_final.predict(validate_x)
    y_hat_oos = model_final.predict(oos_x)
    print(network)
    print('R^2: %.8f' % (r2_score(validate_y, pred_val)))
    print('MSE: %.8f' % (mean_squared_error(validate_y, pred_val)))
    predictions.iloc[network,:] = [mean_squared_error(validate_y, pred_val), y_hat_oos]

  sorted_predictions = predictions.sort_values(by='score', ascending=True)
  y_hat = pd.DataFrame(mean(sorted_predictions.iloc[0:3,1]))
  y_hat.index = oos_y.index
  y_hat.columns = ['Predictions']

  # Fit model
  # print('Nr of trees: ', n_estimators_final)
  # print('Max Depth: ', max_depth_final)
  # print('Max Features: ', max_features_final)
  # model_final = RandomForestRegressor(n_estimators=n_estimators_final,
  #                                    max_depth=max_depth_final,
  #                                    max_features=max_features_final)
  # model_final.fit(total_train_x, total_train_y)
  # y_hat = pd.DataFrame(model_final.predict(oos_x))

  print('Yearly:',r2_score(oos_y, y_hat))
  oos_pred_slope = pd.concat([oos_pred_slope, y_hat], ignore_index=True) #oos_pred_level = oos_pred_level.append(y_hat)
  print('Total:',r2_score(total_oos[IV], oos_pred_slope))
  oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/slope_predictions_NN_1_progress.csv', index=True)

oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/slope_predictions_NN_1.csv', index=True)


# Curvature

In [ ]:
trainsize = 11
validatesize = 5
oossize = 1

def isolate(data):
  data = data.iloc[:,1:]
  return data
oos_pred_curve   = isolate(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/curve_predictions_NN_progress_yearly.csv'))

window = 2018
train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)

train=402892, validate=164247, oos=30002


In [11]:
from sklearn.inspection import permutation_importance
# Complete number of observations is 982285
# We take the initial train, validate and oos sample to be 8, 5 and 13 years respectively (Ratios similar to Gu Kelly Xiu)
# On average one month contains 3000 options, therefore we loop over 12 * 3000 options, which is similar to looping over a year
trainsize = 11
validatesize = 5
oossize = 1

# Initialise predictionstring
IV = 'curve'
oos_pred_curve = pd.DataFrame()

# #start of last prediction
# def isolate(data):
#   data = data.iloc[:,1:]
#   return data
# oos_pred_curve   = isolate(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/curve_predictions_NN_2_progress_yearly.csv'))
# oos_pred_curve.columns = ['Predictions']

# define the model
def create_model():
  model = Sequential()
  model.add(Dense(units=32,
                  activation='relu',
                  input_dim= train_x.shape[1],
                  bias_regularizer= regularizers.L2(1e-4),
                  activity_regularizer= regularizers.L2(1e-6)
                  ))
  model.add(Dense(units=32, activation= 'relu',
                  bias_regularizer=regularizers.L2(1e-4),
                  activity_regularizer=regularizers.L2(1e-6) #activation
                  ))
  model.add(Dense(units=1, activation= 'linear')) #miss linear

  # compile the model
  model.compile(optimizer= Adam(), # was 0.01
                loss= 'mean_squared_error',
                metrics=['mse'])
  return model

window = 2021
train, validate, oos, total_train, total_oos = samplesplitting(data_curve, trainsize, validatesize, oossize, window)
train_x, train_y              = split_xy(train,IV)
validate_x, validate_y        = split_xy(validate,IV)
oos_x, oos_y                  = split_xy(oos,IV)
total_train_x, total_train_y  = split_xy(total_train,IV)
feature_importance = np.zeros(total_train_x.shape[1]) #new

# Parameter_Trials={'batch_size':[100],
#                       'epochs':[50],
#                       'bias':  [1e-4] ,
#                       'activity':  [1e-6]
                    #'Optimizer_trial':['rmsprop'], #'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'
                  #'activation': ['linear']#['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
                  # 'units1': [64, 32],
                  # 'units2': [32, 16]
                #  } #learn rate, momentum

# RegModel=KerasRegressor(create_model, verbose=0)
# grid_search=GridSearchCV(estimator=RegModel, param_grid=Parameter_Trials, cv=2, verbose = 10)
# model_final = grid_search.fit(total_train_x, total_train_y)
# best_params = grid_search.best_params_
# print("Best Hyperparameters:", best_params)

model_final = KerasRegressor(create_model, epochs=50, batch_size=100, verbose=0)

for window in range(2012, 2021+1):
  print('Iteration: ', window)

  # Sample splitting
  train, validate, oos, total_train, total_oos = samplesplitting(data_curve, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  predictions = pd.DataFrame(index=range(5), columns=['score', 'oos_prediction'])
  feature_importance_networks = pd.DataFrame(index=range(5), columns=['score', 'importances']) #new

  for network in range(0,5):
    # model=KerasRegressor(create_model, verbose=0)
    model_final.fit(total_train_x, total_train_y)

    # Calculate feature importance using permutation importance #NEW
    results = permutation_importance(model_final, total_train_x, total_train_y, n_repeats=1, random_state=42) #NEW
    # Get the feature importances and feature names
    importance = results.importances_mean #NEW

    pred_val = model_final.predict(validate_x)
    y_hat_oos = model_final.predict(oos_x)
    print(network)
    print('R^2: %.8f' % (r2_score(validate_y, pred_val)))
    print('MSE: %.8f' % (mean_squared_error(validate_y, pred_val)))
    predictions.iloc[network,:] = [mean_squared_error(validate_y, pred_val), y_hat_oos]
    feature_importance_networks.iloc[network,:] = [mean_squared_error(validate_y, pred_val), importance] #NEW

  sorted_predictions = predictions.sort_values(by='score', ascending=True)
  y_hat = pd.DataFrame(mean(sorted_predictions.iloc[0:2,1]))
  y_hat.index = oos_y.index
  y_hat.columns = ['Predictions']

  # Obtain feature importance
  sorted_importance = feature_importance_networks.sort_values(by='score', ascending=True) #NEW
  model_feat_imp = pd.DataFrame(mean(sorted_importance.iloc[0:2,1])) #NEW
  feature_importance += np.array(model_feat_imp).flatten() #NEW

  # Fit model
  # print('Nr of trees: ', n_estimators_final)
  # print('Max Depth: ', max_depth_final)
  # print('Max Features: ', max_features_final)
  # model_final = RandomForestRegressor(n_estimators=n_estimators_final,
  #                                    max_depth=max_depth_final,
  #                                    max_features=max_features_final)
  # model_final.fit(total_train_x, total_train_y)
  # y_hat = pd.DataFrame(model_final.predict(oos_x))

  print('Yearly:',r2_score(oos_y, y_hat))
  oos_pred_curve = pd.concat([oos_pred_curve, y_hat], ignore_index=True) #oos_pred_level = oos_pred_level.append(y_hat)
  print('Total:',r2_score(total_oos[IV], oos_pred_curve))
  oos_pred_curve.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/curve_predictions_NN_2_progress_yearly.csv', index=True)

feature_importance_mean = feature_importance / 10 #NEW
np.savetxt('/content/drive/MyDrive/Msc Thesis/code/Results/NN/feature_importance_mean_curve_2_yearly.csv', feature_importance_mean, delimiter=',') #NEW

oos_pred_curve.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/curve_predictions_NN_2_yearly.csv', index=True)


train=504261, validate=148707, oos=27899
Iteration:  2012


<ipython-input-11-9e6ac0dd179f>:65: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_final = KerasRegressor(create_model, epochs=50, batch_size=100, verbose=0)


train=210839, validate=158189, oos=33864
0
R^2: 0.05157142
MSE: 0.00597253
1
R^2: 0.05618201
MSE: 0.00594350
2
R^2: 0.04411384
MSE: 0.00601950
3
R^2: 0.06004548
MSE: 0.00591917
4
R^2: 0.05089596
MSE: 0.00597679
Yearly: -0.011041960626488878
Total: -0.011041960626488878
Iteration:  2013
train=241135, validate=161757, oos=34182
0
R^2: 0.06133028
MSE: 0.00769562
1
R^2: 0.05939688
MSE: 0.00771147
2
R^2: 0.05769541
MSE: 0.00772542
3
R^2: 0.06091107
MSE: 0.00769905
4
R^2: 0.05802547
MSE: 0.00772271
Yearly: 0.0006214541261077011
Total: -0.002880379301016589
Iteration:  2014
train=272284, validate=164790, oos=34113
0
R^2: 0.04808857
MSE: 0.00876703
1
R^2: 0.04481384
MSE: 0.00879719
2
R^2: 0.05456316
MSE: 0.00870740
3
R^2: 0.05345913
MSE: 0.00871756
4
R^2: 0.04716476
MSE: 0.00877553
Yearly: -0.03053999998087842
Total: -0.0060252505575266735
Iteration:  2015
train=303150, validate=168037, oos=33074
0
R^2: 0.06810979
MSE: 0.01073017
1
R^2: 0.06715513
MSE: 0.01074116
2
R^2: 0.07112566
MSE: 0.01069

In [ ]:
#start of last prediction
def isolate(data):
  data = data.iloc[:,1:]
  return data
oos_pred_curve   = isolate(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/Results/NN/curve_predictions_NN_1_progress.csv'))

print(oos_pred_curve)

        Predictions
0          0.042977
1          0.063609
2          0.034104
3          0.034903
4          0.034587
...             ...
191995     0.065792
191996     0.027477
191997     0.075264
191998     0.085852
191999     0.067550

[192000 rows x 1 columns]
